#注意:
该文件就是实体识别那部分的文件，放在这里仅供参考，用来构建关键词提取-推理.ipynb,直接运行的跑不起来的，基于的数据集不一样，batch_yield部分会报错。

In [1]:
#colab中运行jupyter文件的步骤：
# 1.挂载云盘
from google.colab import drive
drive.mount('/content/gdrive')

# 2.安装需要的软件
!pip3 install transformers
!pip3 install pytorch-crf

import os
def get_root_dir():
    if os.path.exists('/content/gdrive/MyDrive/第二次完成面试机器人_toColab/src/keyword'):
        return '/content/gdrive/MyDrive/第二次完成面试机器人_toColab/src/keyword/'
    else:
        return './' #在本地

# 3.调用系统命令，切换到对应工程路径，相当于cd，但是直接!cd是不行的
print("path:",get_root_dir())
os.chdir(get_root_dir())

# 4.再次确认路径
!pwd
!ls

Mounted at /content/gdrive
     |████████████████████████████████| 3.4 MB 12.9 MB/s 
     |████████████████████████████████| 3.3 MB 39.1 MB/s 
     |████████████████████████████████| 61 kB 443 kB/s 
     |████████████████████████████████| 596 kB 45.7 MB/s 
     |████████████████████████████████| 895 kB 43.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
path: /content/gdrive/MyDrive/第二次完成面试机器人_toColab/src/keyword/
/content/gdrive/MyDrive/第二次完成面试机器人_toColab/src/keyword
bert_crf.h5   parameter.pkl	  __pycache__	    关键词提取-推理.ipynb
model_zoo.py  prev_trained_model  关键词提取.ipynb  测试模型.ipynb


In [2]:
from collections import defaultdict
from operator import itemgetter
import numpy as np
import torch
import torch.nn.functional as F # pytorch 激活函数的类
import pickle as pk
import pandas as pd
from tqdm import tqdm
from model_zoo import * #模型定义

def load_model(model_name):
    parameter = pk.load(open('parameter.pkl','rb'))
#     parameter['device'] = torch.device('cpu')
    if 'bert' in model_name:
        if 'speed' in model_name:
            model = eval(model_name.split('-')[0]+"(config,parameter).to(parameter['device'])")
        else:
            model = eval(model_name+"(config,parameter).to(parameter['device'])")
    else:
        model = eval(model_name+"(parameter).to(parameter['device'])")
    model.load_state_dict(torch.load(model_name+'.h5'))
    model.eval() 
    return model,parameter

def list2torch(ins):
    return torch.from_numpy(np.array(ins))

def batch_yield(parameter,shuffle = True,isTrain = True,isBert = False):
    data_set = parameter['data_set']['train'] if isTrain else parameter['data_set']['dev']
    Epoch = parameter['epoch'] if isTrain else 1
    for epoch in range(Epoch):
        # 每轮对原始数据进行随机化
        if shuffle:
            random.shuffle(data_set)
        inputs,targets = [],[]
        max_len = 0
        for items in tqdm(data_set):
            if not isBert:
                input = itemgetter(*items[0])(parameter['word2ind'])
                input = input if type(input) == type(()) else (input,0)
            else:
                input = tokenizer.convert_tokens_to_ids(items[0])
            target = itemgetter(*items[1])(parameter['key2ind'])
            target = target if type(target) == type(()) else (target,0)
            if len(input) > max_len:
                max_len = len(input)
            inputs.append(list(input))
            targets.append(list(target))
            if len(inputs) >= parameter['batch_size']:
                inputs = [i+[0]*(max_len-len(i)) for i in inputs]
                targets = [i+[-1]*(max_len-len(i)) for i in targets]
                yield list2torch(inputs),list2torch(targets),None,False
                inputs,targets = [],[]
                max_len = 0
        inputs = [i+[0]*(max_len-len(i)) for i in inputs]
        targets = [i+[-1]*(max_len-len(i)) for i in targets]
        yield list2torch(inputs),list2torch(targets),epoch,False
        inputs,targets = [],[]
        max_len = 0
    yield None,None,None,True


def eval_model(model_name):
    model,parameter = load_model(model_name)
    count_table = {}
    if 'bert' not in model_name:
        test_yield = batch_yield(parameter,shuffle = False,isTrain = False)
    else:
        test_yield = batch_yield(parameter,shuffle = False,isTrain = False,isBert = True)
    while 1:
        inputs,targets,_,keys = next(test_yield)
        if not keys:
            pred = model(inputs.long().to(parameter['device']))
            if 'crf' in model_name:
                pred = model(inputs.long().to(parameter['device']))
                predicted_index = np.array(model.crf.decode(pred))
                targets = targets.numpy()#.long().to(parameter['device'])
            else:
                predicted_prob,predicted_index = torch.max(F.softmax(pred, 1), 1)
                predicted_index = predicted_index.reshape(inputs.shape)
                targets = targets.long().to(parameter['device'])
            right = (targets == predicted_index)
            for i in range(1,parameter['output_size']):
                if i not in count_table:
                    count_table[i] = {
                    'pred':len(predicted_index[(predicted_index == i) & (targets != -1)]),
                    'real':len(targets[targets == i]),
                    'common':len(targets[right & (targets == i)])
                    }
                else:
                    count_table[i]['pred'] += len(predicted_index[predicted_index == i])
                    count_table[i]['real'] += len(targets[targets == i])
                    count_table[i]['common'] += len(targets[right & (targets == i)])
        else:
            break
    count_pandas = {}
    name,count = list(parameter['key2ind'].keys())[1:],list(count_table.values())
    for ind,i in enumerate(name):
        i = i.split('-')[1]
        if i in count_pandas:
            count_pandas[i][0] += count[ind]['pred']
            count_pandas[i][1] += count[ind]['real']
            count_pandas[i][2] += count[ind]['common']
        else:
            count_pandas[i] = [0,0,0]
            count_pandas[i][0] = count[ind]['pred']
            count_pandas[i][1] = count[ind]['real']
            count_pandas[i][2] = count[ind]['common']
    count_pandas['all'] = [sum([count_pandas[i][0] for i in count_pandas]),
                      sum([count_pandas[i][1] for i in count_pandas]),
                      sum([count_pandas[i][2] for i in count_pandas])]
    name = count_pandas.keys()
    count_pandas = pd.DataFrame(count_pandas.values())
    count_pandas.columns = ['pred','real','common']
    count_pandas['p'] = count_pandas['common']/count_pandas['pred']
    count_pandas['r'] = count_pandas['common']/count_pandas['real']
    count_pandas['f1'] = 2*count_pandas['p']*count_pandas['r']/(count_pandas['p']+count_pandas['r'])
    count_pandas.index = list(name)
    return count_pandas

In [6]:
eval_model('bilstm')

FileNotFoundError: ignored

In [7]:
eval_model('bilstm_crf')

FileNotFoundError: ignored

In [ ]:
eval_model('bert')

100%|█████████████████████████████████████████████████████████████████████████████| 1343/1343 [00:03<00:00, 388.70it/s]


,pred,real,common,p,r,f1
name,1515,1486,1316,0.868647,0.885599,0.877041
company,1795,1693,1412,0.786630,0.834022,0.809633
game,1818,1657,1531,0.842134,0.923959,0.881151
organization,1436,1454,1083,0.754178,0.744842,0.749481
movie,1000,1043,920,0.920000,0.882071,0.900636
address,1740,1702,1269,0.729310,0.745593,0.737362
position,1236,1201,984,0.796117,0.819317,0.807550
government,1461,1315,1171,0.801506,0.890494,0.843660
scene,902,931,702,0.778271,0.754028,0.765957
book,988,1031,893,0.903846,0.866149,0.884596


In [3]:
eval_model('bert_crf')

KeyError: ignored